In [2]:
# Imports
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration
import optuna
import os
import json
from tqdm import tqdm
import numpy as np
import time
import shutil

2025-04-15 21:44:53.707352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744753494.002555      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744753494.091067      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"Number of GPUs available: {torch.cuda.device_count()}")

# Load dataset
with open('/kaggle/input/moe-dataset/combined_scientific_papers.json', 'r') as f:
    data = json.load(f)

# Domain to label mapping
domain_to_label = {domain: idx for idx, domain in enumerate(set(entry['domain'] for entry in data))}
num_labels = len(domain_to_label)
num_experts = num_labels
print(f"Number of experts/domains: {num_experts}")

Using device: cuda
Number of GPUs available: 2
Number of experts/domains: 3


In [4]:
class ScientificDataset(Dataset):
    def __init__(self, data, domain_to_label):
        self.queries = [entry['text'][:100] for entry in data]
        self.labels = [domain_to_label[entry['domain']] for entry in data]
        self.responses = [entry['text'] for entry in data]
    
    def __len__(self):
        return len(self.queries)
    
    def __getitem__(self, idx):
        return self.queries[idx], self.labels[idx], self.responses[idx]

# Collate functions
def gating_collate_fn(batch):
    queries, labels, _ = zip(*batch)
    tokenized = bert_tokenizer(list(queries), padding=True, truncation=True, return_tensors='pt')
    return tokenized, torch.tensor(labels)

def expert_collate_fn(batch):
    queries, responses = zip(*batch)
    inputs = t5_tokenizer(list(queries), padding=True, truncation=True, return_tensors='pt')
    targets = t5_tokenizer(list(responses), padding=True, truncation=True, return_tensors='pt')
    return inputs, targets['input_ids']

# Expert dataset
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Extract expert data
def get_expert_data(dataset, expert_id):
    return [(query, response) for query, label, response in dataset if label == expert_id]

# Training functions with gradient accumulation
def train_gating_model(model, train_loader, val_loader, lr, epochs, accumulation_steps=4):
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    model = nn.DataParallel(model)
    model.to(device)
    loss_fct = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            outputs = model(input_ids=inputs['input_ids'], 
                            attention_mask=inputs['attention_mask'])
            logits = outputs.logits
            loss = loss_fct(logits, labels) / accumulation_steps
            loss.backward()
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            total_loss += loss.item() * accumulation_steps
        avg_loss = total_loss / len(train_loader)
        print(f"Gating Epoch {epoch+1}, Train Loss: {avg_loss:.4f}")
        gc.collect()
        torch.cuda.empty_cache()
    return model

def train_expert(model, train_loader, val_loader, lr, epochs, expert_id, accumulation_steps=4):
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    model = nn.DataParallel(model)
    model.to(device)
    pad_token_id = model.module.config.pad_token_id
    loss_fct = nn.CrossEntropyLoss(ignore_index=pad_token_id)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            decoder_input_ids = labels[:, :-1].clone()
            target_labels = labels[:, 1:].clone()
            outputs = model(input_ids=inputs['input_ids'], 
                            attention_mask=inputs['attention_mask'], 
                            decoder_input_ids=decoder_input_ids)
            logits = outputs.logits
            loss = loss_fct(logits.view(-1, logits.size(-1)), target_labels.view(-1)) / accumulation_steps
            loss.backward()
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            total_loss += loss.item() * accumulation_steps
        avg_loss = total_loss / len(train_loader)
        print(f"Expert {expert_id} Epoch {epoch+1}, Train Loss: {avg_loss:.4f}")
        gc.collect()
        torch.cuda.empty_cache()
    return model

# Evaluation functions
def evaluate_gating_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            outputs = model(input_ids=inputs['input_ids'], 
                           attention_mask=inputs['attention_mask'])
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def evaluate_expert(model, val_loader):
    if len(val_loader) == 0:
        return 0.0
    model.eval()
    total_loss = 0
    pad_token_id = model.module.config.pad_token_id
    loss_fct = nn.CrossEntropyLoss(ignore_index=pad_token_id)
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            decoder_input_ids = labels[:, :-1].clone()
            target_labels = labels[:, 1:].clone()
            outputs = model(input_ids=inputs['input_ids'], 
                            attention_mask=inputs['attention_mask'], 
                            decoder_input_ids=decoder_input_ids)
            logits = outputs.logits
            loss = loss_fct(logits.view(-1, logits.size(-1)), target_labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Optuna objective
def objective(trial):
    trial_dir = f'trial_{trial.number}'
    os.makedirs(trial_dir, exist_ok=True)
    
    gating_lr = trial.suggest_float('gating_lr', 1e-5, 1e-3, log=True)
    gating_epochs = trial.suggest_int('gating_epochs', 3, 10)
    expert_lr = trial.suggest_float('expert_lr', 1e-5, 1e-3, log=True)
    expert_epochs = trial.suggest_int('expert_epochs', 3, 10)
    
    gating_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_experts).to(device)
    gating_model = train_gating_model(gating_model, train_loader_gating, val_loader_gating, gating_lr, gating_epochs)
    gating_accuracy = evaluate_gating_model(gating_model, val_loader_gating)
    
    gating_model_path = os.path.join(trial_dir, 'gating.pt')
    torch.save(gating_model.state_dict(), gating_model_path)
    gating_model.to('cpu')
    torch.cuda.empty_cache()
    del gating_model
    
    experts_paths = {}
    expert_losses = []
    for expert_id in range(num_experts):
        train_data = train_expert_data[expert_id]
        val_data = val_expert_data[expert_id]
        if not train_data:
            continue
        expert_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
        train_dataset_expert = TextDataset(train_data)
        val_dataset_expert = TextDataset(val_data)
        train_loader_expert = DataLoader(train_dataset_expert, batch_size=2, shuffle=True, collate_fn=expert_collate_fn)
        val_loader_expert = DataLoader(val_dataset_expert, batch_size=2, shuffle=False, collate_fn=expert_collate_fn)
        expert_model = train_expert(expert_model, train_loader_expert, val_loader_expert, expert_lr, expert_epochs, expert_id)
        loss = evaluate_expert(expert_model, val_loader_expert)
        expert_losses.append(loss)
        
        expert_path = os.path.join(trial_dir, f'expert_{expert_id}.pt')
        torch.save(expert_model.state_dict(), expert_path)
        experts_paths[expert_id] = expert_path
        expert_model.to('cpu')
        torch.cuda.empty_cache()
        del expert_model
    
    avg_expert_loss = sum(expert_losses) / len(expert_losses) if expert_losses else 0
    combined_metric = gating_accuracy - 0.1 * avg_expert_loss
    
    trial.set_user_attr('gating_accuracy', gating_accuracy)
    trial.set_user_attr('avg_expert_loss', avg_expert_loss)
    trial.set_user_attr('trial_dir', trial_dir)
    trial.set_user_attr('experts_paths', experts_paths)
    
    return combined_metric

In [5]:
if __name__ == "__main__":
    full_dataset = ScientificDataset(data, domain_to_label)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    t5_tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)
    
    train_loader_gating = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=gating_collate_fn)
    val_loader_gating = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=gating_collate_fn)
    
    train_expert_data = {i: get_expert_data(train_dataset, i) for i in range(num_experts)}
    val_expert_data = {i: get_expert_data(val_dataset, i) for i in range(num_experts)}
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)
    
    print("\nBenchmarking Top 5 Configurations:")
    top_trials = sorted(study.trials, key=lambda t: t.value, reverse=True)[:5]
    for i, trial in enumerate(top_trials):
        print(f"\nMOE {i+1}:")
        print(f"  Trial Number: {trial.number}")
        print(f"  Combined Metric: {trial.value:.4f}")
        print(f"  Gating Accuracy: {trial.user_attrs['gating_accuracy']:.4f}")
        print(f"  Avg Expert Loss: {trial.user_attrs['avg_expert_loss']:.4f}")
        print(f"  Hyperparameters: {trial.params}")
    
    for i, trial in enumerate(top_trials):
        moe_dir = f'MOE_{i+1}'
        os.makedirs(moe_dir, exist_ok=True)
        trial_dir = trial.user_attrs['trial_dir']
        
        shutil.copy(os.path.join(trial_dir, 'gating.pt'), os.path.join(moe_dir, 'gating.pt'))
        
        experts_paths = trial.user_attrs['experts_paths']
        for expert_id, expert_path in experts_paths.items():
            shutil.copy(expert_path, os.path.join(moe_dir, f'expert_{expert_id}.pt'))
        
        metrics = {
            'gating_accuracy': trial.user_attrs['gating_accuracy'],
            'avg_expert_loss': trial.user_attrs['avg_expert_loss'],
            'combined_metric': trial.value
        }
        with open(os.path.join(moe_dir, 'metrics.json'), 'w') as f:
            json.dump(metrics, f)
        with open(os.path.join(moe_dir, 'hyperparams.json'), 'w') as f:
            json.dump(trial.params, f)
    
    top_trial_numbers = [trial.number for trial in top_trials]
    for trial in study.trials:
        if trial.number not in top_trial_numbers:
            trial_dir = trial.user_attrs.get('trial_dir')
            if trial_dir and os.path.exists(trial_dir):
                shutil.rmtree(trial_dir)
    
    print("\nTop 5 MOE models saved.")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

Gating Epoch 1, Train Loss: 1.0588
Gating Epoch 2, Train Loss: 0.9437
Gating Epoch 3, Train Loss: 1.0575
Gating Epoch 4, Train Loss: 1.1194
Gating Epoch 5, Train Loss: 1.0354
Gating Epoch 6, Train Loss: 1.0459
Gating Epoch 7, Train Loss: 1.0716
Gating Epoch 8, Train Loss: 1.1385
Gating Epoch 9, Train Loss: 1.2019
Gating Epoch 10, Train Loss: 1.1667
Expert 0 Epoch 1, Train Loss: 8.4275
Expert 0 Epoch 2, Train Loss: 8.0243
Expert 0 Epoch 3, Train Loss: 7.7832
Expert 0 Epoch 4, Train Loss: 7.9420
Expert 0 Epoch 5, Train Loss: 7.7846
Expert 0 Epoch 6, Train Loss: 8.3761
Expert 0 Epoch 7, Train Loss: 8.4376
Expert 0 Epoch 8, Train Loss: 8.1266
Expert 1 Epoch 1, Train Loss: 9.0074
Expert 1 Epoch 2, Train Loss: 5.7064
Expert 1 Epoch 3, Train Loss: 5.5980
Expert 1 Epoch 4, Train Loss: 5.3785
Expert 1 Epoch 5, Train Loss: 5.3279
Expert 1 Epoch 6, Train Loss: 5.1897
Expert 1 Epoch 7, Train Loss: 5.0970
Expert 1 Epoch 8, Train Loss: 4.9211
Expert 2 Epoch 1, Train Loss: 9.2033
Expert 2 Epoch 2, Tr

[I 2025-04-15 21:54:16,491] Trial 16 finished with value: -0.08947065671284993 and parameters: {'gating_lr': 0.0002121563783901395, 'gating_epochs': 10, 'expert_lr': 0.0008240674624446471, 'expert_epochs': 8}. Best is trial 16 with value: -0.08947065671284993.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Gating Epoch 1, Train Loss: 1.2795
Gating Epoch 2, Train Loss: 1.1124
Gating Epoch 3, Train Loss: 1.0275
Gating Epoch 4, Train Loss: 1.0027
Gating Epoch 5, Train Loss: 1.2186
Gating Epoch 6, Train Loss: 1.2887
Gating Epoch 7, Train Loss: 1.1691
Gating Epoch 8, Train Loss: 1.1721
Gating Epoch 9, Train Loss: 1.2138
Gating Epoch 10, Train Loss: 1.2679
Expert 0 Epoch 1, Train Loss: 8.8489
Expert 0 Epoch 2, Train Loss: 7.8500
Expert 0 Epoch 3, Train Loss: 8.3981
Expert 0 Epoch 4, Train Loss: 8.5941
Expert 0 Epoch 5, Train Loss: 8.1092
Expert 0 Epoch 6, Train Loss: 8.2189
Expert 0 Epoch 7, Train Loss: 7.7611
Expert 0 Epoch 8, Train Loss: 8.7595
Expert 1 Epoch 1, Train Loss: 8.1021
Expert 1 Epoch 2, Train Loss: 5.9502
Expert 1 Epoch 3, Train Loss: 5.5623
Expert 1 Epoch 4, Train Loss: 5.4477
Expert 1 Epoch 5, Train Loss: 5.3773
Expert 1 Epoch 6, Train Loss: 5.1596
Expert 1 Epoch 7, Train Loss: 5.1114
Expert 1 Epoch 8, Train Loss: 5.0824
Expert 2 Epoch 1, Train Loss: 8.7942
Expert 2 Epoch 2, Tr

[W 2025-04-15 21:54:52,807] Trial 17 failed with parameters: {'gating_lr': 8.249007070428141e-05, 'gating_epochs': 10, 'expert_lr': 0.0009826494734997409, 'expert_epochs': 8} because of the following error: RuntimeError('[enforce fail at inline_container.cc:603] . unexpected pos 224245888 vs 224245776').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 850, in save
    _save(
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1114, in _save
    zip_file.write_record(name, storage, num_bytes)
RuntimeError: [enforce fail at inline_container.cc:778] . PytorchStreamWriter failed writing file data/86: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/i

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 224245888 vs 224245776